# MLRun Job Example

### Setup MLRun

In [1]:
import os
from os import path
from mlrun import set_environment, new_project, mlconf, code_to_function, mount_v3io, get_run_db

# Set the default environment configuration
project_name, artifact_path = set_environment(
    project="igz-onboarding",
    artifact_path="/User/artifacts/{{run.project}}/{{workflow.uid}}"
)

# Create project
project_path = path.abspath("project")
project = new_project(name=project_name, context=project_path)

print(f"Project name: {project_name}\nProject path: {project_path}")
print(f"Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Project name: igz-onboarding
Project path: /User/igz_repos/igz-onboarding/project
Artifacts path: /User/artifacts/{{run.project}}/{{workflow.uid}}
MLRun DB path: http://mlrun-api:8080


### Build Docker Image

In [2]:
image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/new-image"
image

'docker-registry.default-tenant.app.us-sales-fs.iguazio-cd0.com:80/new-image'

In [3]:
# # Build Docker Image (only needs to be run once)
# build_image = new_function(name="build-image", kind="job")
# build_image.build_config(
#     image=image, base_image="mlrun/ml-models-gpu", commands=["pip install v3io_frames"]
# )
# build_image.deploy(with_mlrun=False)

### Define MLRun Function

In [4]:
# Create MLRun function from Python file
fn = code_to_function(
    name="training-logging-demo",
    project=project_name,
    filename="src_code/mlrun_data_model_fn.py",
    handler="handler",
    kind='job',
    image="mlrun/ml-models"
).apply(mount_v3io())

# Assign resources
fn.with_requests(cpu=1, mem="1G")
fn.with_limits(cpu=2, mem="2G")

### Run MLRun Function Locally

In [5]:
run = fn.run(inputs={
                'model' : f"{os.path.abspath('.')}/assets/model.h5",
                "csv" : f"{os.path.abspath('.')}/assets/heart.csv"
             },
             artifact_path=artifact_path,
             local=True)

> 2021-05-13 14:38:13,029 [info] starting run training-logging-demo-handler uid=24cddb5112b94aa38c574ac61efd54d1 DB=http://mlrun-api:8080
> 2021-05-13 14:38:14,413 [info] Logging model
> 2021-05-13 14:38:14,516 [info] Logging CSV


Converting input from bool to <class 'numpy.uint8'> for compatibility.


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
igz-onboarding,...1efd54d1,0,May 13 14:38:14,completed,training-logging-demo-handler,v3io_user=nickkind=owner=nickhost=nick-jupyter-8b7d7474c-sdlr6,modelcsv,,,my_modelmy_csv


to track results use .show() or .logs() or in CLI: 
!mlrun get run 24cddb5112b94aa38c574ac61efd54d1 --project igz-onboarding , !mlrun logs 24cddb5112b94aa38c574ac61efd54d1 --project igz-onboarding
> 2021-05-13 14:38:14,753 [info] run executed, status=completed


### Run MLRun Function on Cluster

In [6]:
run = fn.run(inputs={
                'model' : f"{os.path.abspath('.')}/assets/model.h5",
                "csv" : f"{os.path.abspath('.')}/assets/heart.csv"
             },
             artifact_path=artifact_path)

> 2021-05-13 14:38:14,758 [info] starting run training-logging-demo-handler uid=fe1b73c90aee4984853658df58711b65 DB=http://mlrun-api:8080
> 2021-05-13 14:38:14,923 [info] Job is running in the background, pod: training-logging-demo-handler-kz9v4
2021-05-13 14:38:20.315881: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-05-13 14:38:20.322649: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2499995000 Hz
2021-05-13 14:38:20.323516: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560f0bf2d840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-13 14:38:20.323544: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-05-13 14:38:20.323656: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setti

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
igz-onboarding,...58711b65,0,May 13 14:38:20,completed,training-logging-demo-handler,v3io_user=nickkind=jobowner=nickhost=training-logging-demo-handler-kz9v4,modelcsv,,,my_modelmy_csv


to track results use .show() or .logs() or in CLI: 
!mlrun get run fe1b73c90aee4984853658df58711b65 --project igz-onboarding , !mlrun logs fe1b73c90aee4984853658df58711b65 --project igz-onboarding
> 2021-05-13 14:38:21,053 [info] run executed, status=completed


### View Run Logs

In [7]:
db = get_run_db()
resp = db.watch_log(run.uid(), project=project_name)

2021-05-13 14:38:20.315881: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-05-13 14:38:20.322649: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2499995000 Hz
2021-05-13 14:38:20.323516: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560f0bf2d840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-13 14:38:20.323544: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-05-13 14:38:20.323656: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
> 2021-05-13 14:38:20,378 [info] Logging model
> 2021-05-13 14:38:20,473 [info] Logging CSV
> 2021-05-13 14:38:20,629 [info] run executed, status=completed
Converting input from